In [10]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import geopandas as gpd

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes('-topmost', True)

selected_dir = filedialog.askdirectory(title="Wybierz folder z plikami CSV")

if not selected_dir:
    print("Operacja anulowana: Nie wybrano folderu.")
else:
    dir_path = Path(selected_dir)
    csv_files = list(dir_path.glob("*.csv"))
    
    print(f"Znaleziono plików CSV do przetworzenia: {len(csv_files)}")
    
    df_list = []

    for i, filepath in enumerate(csv_files, 1):
        print(f"Wczytywanie pliku {i}/{len(csv_files)}: {filepath.name} ...")
        try:
            df = pd.read_csv(filepath, index_col=None, header=0)
            df_list.append(df)
        except Exception as e:
            print(f"Błąd przy wczytywaniu {filepath.name}: {e}")
    if df_list:
        print("Łączenie danych w jedną tabelę (to może chwilę potrwać)...")
        df_out = pd.concat(df_list, ignore_index=True)
        print(f"Gotowe! Ostateczna ramka danych ma wymiary: {df_out.shape}")
    else:
        print("Nie udało się załadować żadnych danych.")

Znaleziono plików CSV do przetworzenia: 11
Wczytywanie pliku 1/11: apartments_pl_2023_08.csv ...
Wczytywanie pliku 2/11: apartments_pl_2023_09.csv ...
Wczytywanie pliku 3/11: apartments_pl_2023_10.csv ...
Wczytywanie pliku 4/11: apartments_pl_2023_11.csv ...
Wczytywanie pliku 5/11: apartments_pl_2023_12.csv ...
Wczytywanie pliku 6/11: apartments_pl_2024_01.csv ...
Wczytywanie pliku 7/11: apartments_pl_2024_02.csv ...
Wczytywanie pliku 8/11: apartments_pl_2024_03.csv ...
Wczytywanie pliku 9/11: apartments_pl_2024_04.csv ...
Wczytywanie pliku 10/11: apartments_pl_2024_05.csv ...
Wczytywanie pliku 11/11: apartments_pl_2024_06.csv ...
Łączenie danych w jedną tabelę (to może chwilę potrwać)...
Gotowe! Ostateczna ramka danych ma wymiary: (195568, 28)


In [5]:
krakow=df_out.loc[df_out["city"]=="krakow"]

In [11]:
krakow_clean=krakow[["price","latitude","longitude","squareMeters"]].copy()
krakow_clean["pricePerMeter"]=krakow["price"]//krakow["squareMeters"]
gdf_krakow_clean=gpd.GeoDataFrame(data=krakow_clean,geometry=gpd.points_from_xy(krakow_clean["longitude"],krakow_clean["latitude"]),crs=4326).to_crs(epsg=2180)


In [20]:
import shapely
target_point=shapely.Point(19.936756,50.061897)
gdf_bufor=gpd.GeoDataFrame(geometry=[target_point],crs="EPSG:4326").to_crs(epsg=2180)
gdf_bufor.geometry=gdf_bufor.geometry.buffer(400)
in_buffer=gpd.sjoin(gdf_krakow_clean,gdf_bufor,how="inner",predicate="within")
mean_price=in_buffer["pricePerMeter"].mean()
mean_price

np.float64(19291.351966873706)